<a href="https://colab.research.google.com/github/matule00/SGE/blob/main/cena_RE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, LeakyReLU, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau, Callback

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 220)

In [11]:
SVR = pd.read_csv('SVR_11_12.csv')
SVR['Time'] = pd.to_datetime(SVR['Time'])

SO = pd.read_csv('odchylka_moje.csv')
SO['Time'] = pd.to_datetime(SO['Time'])

RE = pd.read_csv('RE_cena_moje.csv')
RE['Time'] = pd.to_datetime(RE['Time'])

data = pd.merge(RE, SO, on='Time')

data = pd.merge(data, SVR, on='Time')
data

,Time,aFRR [eur],mFRR+ [eur],mFRR- [eur],mFRR5 [eur],Aktuální odchylka [MW],afrr_p,afrr_m,mfrr_p,mfrr_m,rr_p,rr_m,mfrr5
0,2024-11-28 09:06:00,237.64984,0,0,0,-65.543770,75.02734,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-11-28 09:07:00,187.94983,0,0,0,-42.624687,77.96964,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-11-28 09:08:00,169.51376,0,0,0,-15.013090,76.93359,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-11-28 09:09:00,167.33347,0,0,0,-0.357384,75.89848,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-11-28 09:10:00,167.19435,0,0,0,-10.509396,75.14953,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16187,2024-12-09 14:53:00,123.33817,0,0,0,-121.253532,26.17154,0.0,0.0,0.0,0.0,0.0,0.0
16188,2024-12-09 14:54:00,119.77933,0,0,0,-109.471382,25.58720,0.0,0.0,0.0,0.0,0.0,0.0
16189,2024-12-09 14:55:00,124.77269,0,0,0,-99.722382,22.95162,0.0,0.0,0.0,0.0,0.0,0.0
16190,2024-12-09 14:56:00,121.04552,0,0,0,-107.552216,24.14136,0.0,0.0,0.0,0.0,0.0,0.0
